In [1]:
# set up
import numpy as np
import os

gdb=r"C:\Users\zhu.2906\Documents\ArcGIS\Projects\shrug_india\shrug_india.gdb"
output=r"C:\Users\zhu.2906\Documents\ArcGIS\Projects\shrug_india\output"

arcpy.env.overwriteOutput = True

In [2]:
# drop all things in map
aprx = arcpy.mp.ArcGISProject("CURRENT")
mp = aprx.listMaps("Map")[0]

# Remove all layers
for lyr in mp.listLayers():
 mp.removeLayer(lyr)

Calculate distance from centroid to shoreline

In [16]:
arcpy.analysis.Near("village_2011_coast_centroid",
                    "shoreline_NOAA", None, "LOCATION", "NO_ANGLE", "PLANAR")
arcpy.conversion.TableToTable("village_2011_coast_centroid",
                              output,
                              "village_shoreline_NOAA.csv")

<Result 'C:\\Users\\zhu.2906\\Documents\\ArcGIS\\Projects\\shrug_india\\output\\village_shoreline_NOAA.csv'>

Calculate distance from centroid to bathymetry

generate bathymetry at 100m 50m 40m 30m 20m 10m

In [3]:
# input
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprxMap = aprx.listMaps("Map")[0]
bathymetry=aprxMap.addDataFromPath(r"C:\Users\zhu.2906\OneDrive - The Ohio State University\pythonProject\India_mangroves_python\data\continental shell\ETOPO1_Bed_g_geotiff.tif")


In [11]:
arcpy.ddd.Contour(
    in_raster="ETOPO1_Bed_g_geotiff.tif",
    out_polyline_features="india_bathymetry",
    contour_interval=5,
    base_contour=-100,
    z_factor=1,
    contour_type="CONTOUR",
    max_vertices_per_feature=None
)

<Result 'C:\\Users\\zhu.2906\\Documents\\ArcGIS\\Projects\\shrug_india\\shrug_india.gdb\\india_bathymetry'>

In [13]:
# generate specific contour lines
# for number in [-0,-5,-10,-20,-30,-40,-50,-70,-100]:
for number in [-5]:
    selected=arcpy.management.SelectLayerByAttribute("india_bathymetry", "NEW_SELECTION", "Contour ="+str(number), None)
    arcpy.management.CopyFeatures(selected, "contour_"+str(int(-number)), '', None, None, None)
    



In [14]:
selected=arcpy.management.SelectLayerByAttribute("contour_5", "NEW_SELECTION", "Id=39587033", None)
arcpy.management.CopyFeatures(selected, "contour_5_unique", '', None, None, None)


<Result 'C:\\Users\\zhu.2906\\Documents\\ArcGIS\\Projects\\shrug_india\\shrug_india.gdb\\contour_5_unique'>

In [9]:
# only include the overall line
selected=arcpy.management.SelectLayerByAttribute("contour_0", "NEW_SELECTION", "Id=19784664", None)
arcpy.management.CopyFeatures(selected, "contour_0_unique", '', None, None, None)

selected=arcpy.management.SelectLayerByAttribute("contour_10", "NEW_SELECTION", "Id=19784659", None)
arcpy.management.CopyFeatures(selected, "contour_10_unique", '', None, None, None)

selected=arcpy.management.SelectLayerByAttribute("contour_20", "NEW_SELECTION", "Id=19784649", None)
arcpy.management.CopyFeatures(selected, "contour_20_unique", '', None, None, None)

selected=arcpy.management.SelectLayerByAttribute("contour_30", "NEW_SELECTION", "Id=19784641", None)
arcpy.management.CopyFeatures(selected, "contour_30_unique", '', None, None, None)

selected=arcpy.management.SelectLayerByAttribute("contour_50", "NEW_SELECTION", "Id=19798914", None)
arcpy.management.CopyFeatures(selected, "contour_50_unique", '', None, None, None)

<Result 'C:\\Users\\zhu.2906\\Documents\\ArcGIS\\Projects\\shrug_india\\shrug_india.gdb\\contour_50_unique'>

In [15]:
for number in [-0,-5,-10,-20,-30,-50]:
# village
    arcpy.analysis.Near("village_2011_coast_centroid",
                        "contour_"+str(int(-number))+"_unique", None, "LOCATION", "NO_ANGLE", "PLANAR")
    arcpy.conversion.TableToTable("village_2011_coast_centroid",
                                  output,
                                  "village_contour_"+str(int(-number))+".csv")

get intersection between distance to nearest contour and contour 0

In [3]:
for number in [5,10,20,30,50]:
    number=str(number)

    #generate line
    arcpy.management.XYToLine(
        in_table="village_contour_"+number+".csv",
        out_featureclass="contour_to_line",
        startx_field="X",
        starty_field="Y",
        endx_field="NEAR_X",
        endy_field="NEAR_Y",
        line_type="GEODESIC",
        id_field="index",
        spatial_reference='PROJCS["WGS_1984_India_NSF_LCC",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",4000000.0],PARAMETER["False_Northing",4000000.0],PARAMETER["Central_Meridian",80.0],PARAMETER["Standard_Parallel_1",12.472955],PARAMETER["Standard_Parallel_2",35.17280444444444],PARAMETER["Latitude_Of_Origin",24.0],UNIT["Meter",1.0]];-35091300 -22777300 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision',
        attributes="NO_ATTRIBUTES"
    )
    
    # intersect with 0 line
    arcpy.analysis.Intersect(
        in_features="contour_to_line #;contour_0_unique #",
        out_feature_class=output+'\contour_'+number+'_0_distance.shp',
        join_attributes="ALL",
        cluster_tolerance=None,
        output_type="POINT"
    )

In [ ]:
# intersect with contour 0


# calculate by area

In [2]:
arcpy.management.FeatureToPolygon(
    in_features="contour_20_unique;contour_0_unique;line_2;line_1",
    out_feature_class=r"C:\Users\zhu.2906\Documents\ArcGIS\Projects\shrug_india\shrug_india.gdb\contour_20_0",
    cluster_tolerance=None,
    attributes="ATTRIBUTES",
    label_features=None
)

arcpy.management.FeatureToPolygon(
    in_features="contour_10_unique;contour_0_unique;line_2;line_1",
    out_feature_class=r"C:\Users\zhu.2906\Documents\ArcGIS\Projects\shrug_india\shrug_india.gdb\contour_10_0",
    cluster_tolerance=None,
    attributes="ATTRIBUTES",
    label_features=None
)

arcpy.management.FeatureToPolygon(
    in_features="contour_5_unique;contour_0_unique;line_2;line_1",
    out_feature_class=r"C:\Users\zhu.2906\Documents\ArcGIS\Projects\shrug_india\shrug_india.gdb\contour_5_0",
    cluster_tolerance=None,
    attributes="ATTRIBUTES",
    label_features=None
)

<Result 'C:\\Users\\zhu.2906\\Documents\\ArcGIS\\Projects\\shrug_india\\shrug_india.gdb\\contour_5_0'>

In [1]:
for number in [30, 20,10, 5]:
    
    arcpy.management.FeatureToPolygon(
        in_features="contour_"+str(number)+"_unique;contour_0_unique;line_2;line_1",
        out_feature_class=r"C:\Users\zhu.2906\Documents\ArcGIS\Projects\shrug_india\shrug_india.gdb\contour_"+str(number)+"_0",
        cluster_tolerance=None,
        attributes="ATTRIBUTES",
        label_features=None
    )
    
    arcpy.management.Dissolve(
        in_features="contour_"+str(number)+"_0",
        out_feature_class=r"C:\Users\zhu.2906\Documents\ArcGIS\Projects\shrug_india\shrug_india.gdb\contour_"+str(number)+"_0_d",
        dissolve_field=None,
        statistics_fields=None,
        multi_part="MULTI_PART",
        unsplit_lines="DISSOLVE_LINES",
        concatenation_separator=""
    )

In [3]:
# generate buffer and intersection
for number in [30,20,10, 5]:
    
    # generate buffer for distance from centroid to # meter contours
    arcpy.analysis.Near("village_2011_coast_centroid",
                    "contour_"+str(number)+"_unique", None, "LOCATION", "NO_ANGLE", "PLANAR")

    arcpy.management.XYToLine("village_2011_coast_centroid",
                              "new_village_contour",
                              "X", "Y", "NEAR_X", "NEAR_Y",
                              "GEODESIC",
                              "index",
                              'PROJCS["WGS_1984_India_NSF_LCC",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",4000000.0],PARAMETER["False_Northing",4000000.0],PARAMETER["Central_Meridian",80.0],PARAMETER["Standard_Parallel_1",12.472955],PARAMETER["Standard_Parallel_2",35.17280444444444],PARAMETER["Latitude_Of_Origin",24.0],UNIT["Meter",1.0]]', "NO_ATTRIBUTES")


    # Generate 1 miles buffer around distance to shoreline
    # generate buffer around line and export
    arcpy.analysis.Buffer("new_village_contour", "new_village_contour_"+str(number)+"_buffer", str(1600)+" Meters", "FULL", "ROUND", "NONE", None, "PLANAR")
    arcpy.conversion.FeatureClassToFeatureClass("new_village_contour_"+str(number)+"_buffer",
                                                output,"new_village_contour_"+str(number)+"_buffer.shp")
        